<a href="https://colab.research.google.com/github/shyamsparrow/Jan-G4---NLP-Chatbot/blob/main/Text_Preprocessing_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
%cd /content/MyDrive/AIML/Capstone

[Errno 2] No such file or directory: '/content/MyDrive/AIML/Capstone'
/content


In [31]:
! git clone https://github.com/shyamsparrow/Jan-G4---NLP-Chatbot.git
! git pull

fatal: destination path 'Jan-G4---NLP-Chatbot' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git


In [37]:
import sys
print(os.getcwd())
print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))

/content
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py
/content


In [39]:
_CUSTOM_SUB_CSV_FILE_PATH = os.path.join(os.getcwd(), 'Jan-G4---NLP-Chatbot/data/custom_substitutions.csv')
_CUSTOM_SUB_CSV_FILE_PATH

'/content/Jan-G4---NLP-Chatbot/data/custom_substitutions.csv'

In [33]:
# Standard libraries
!pip install pyspellchecker
import os
import re
import string
import logging
import csv
from pathlib import Path
from functools import wraps
from unicodedata import normalize
from typing import List, Optional, Union, Callable

# Third party libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)

_CUSTOM_SUB_CSV_FILE_PATH = os.path.join(os.path.dirname(__file__), 'data/custom_substitutions.csv')
_IGNORE_SPELLCHECK_WORD_FILE_PATH = os.path.join(os.getcwd(), 'data/ignore_spellcheck_words.txt')


LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)
class PreProcessing:
  def __init__(self, to_lower = False,remove_url=False,remove_special_character=False, remove_punctuation=False,
               remove_whitespace=False,check_spelling=False,remove_stopword=False,substitute_token=False,lemmatize_word=False,
               tokenize_word=False, tokenize_sentence= False):
    
    self.to_lower = to_lower
    self.remove_url=remove_url
    self.remove_special_character=remove_special_character
    self.remove_punctuation=remove_punctuation
    self.remove_whitespace=remove_whitespace
    self.check_spelling=check_spelling
    self.remove_stopword=remove_stopword
    self.substitute_token=substitute_token
    self.lemmatize_word=lemmatize_word
    self.tokenize_word=tokenize_word
    self.tokenize_sentence=tokenize_sentence
  
  def preprocess(self, input_text):

    if self.to_lower:
      input_text = self.to_lower_method(input_text)

    if self.remove_url:
      input_text = self.remove_url_method(input_text)
    
    if self.remove_special_character:
      input_text = self.remove_special_character_method(input_text)
    
    if self.remove_punctuation:
      input_text = self.remove_punctuation_method(input_text)

    if self.remove_whitespace:
      input_text = self.remove_whitespace_method(input_text)
      
    if self.check_spelling:
      input_text = self.check_spelling_method(input_text)
      
    if self.remove_stopword:
      input_text = self.remove_stopword_method(input_text)
      
    if self.substitute_token:
      input_text = self.substitute_token_method(input_text)
      
    if self.lemmatize_word:
      input_text = self.lemmatize_word_method(input_text)
            
    if self.tokenize_word:
      input_text = self.tokenize_word_method(input_text)
            
    if self.tokenize_sentence:
      input_text = self.tokenize_sentence_method(input_text)
    
    if isinstance(input_text, str):
        processed_text = input_text
    else:
        processed_text = ' '.join(input_text)
    return processed_text

    return input_text


  def to_lower_method(self,input_text:str)->str:
    """ Convert input text to lower case """
    return input_text.lower()

  def remove_url_method(self,input_text:str)-> str:
    """ Remove url in the input text """
    return re.sub('(www|http)\S+', '', input_text)

  
  def remove_punctuation_method(self,input_text:str)-> str:
    """
    Removes all punctuations from a string, as defined by string.punctuation or a custom list.
    For reference, Python's string.punctuation is equivalent to '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'
    """
    punctuations = string.punctuation
    processed_text = input_text.translate(str.maketrans('', '', punctuations))
    return processed_text

  def remove_special_character_method(self,input_text:str)-> str:
    """ Removes special characters """
    special_characters = 'å¼«¥ª°©ð±§µæ¹¢³¿®ä£'
    processed_text = input_text.translate(str.maketrans('', '', special_characters))
    return processed_text

  def keep_alpha_numeric_method(self,input_text:str):
    """ Remove any character except alphanumeric characters """
    return ''.join(c for c in input_text if c.isalnum())

  def remove_whitespace_method(self,input_text:str, remove_duplicate_whitespace: bool = True)-> str:
    """ Removes leading, trailing, and (optionally) duplicated whitespace """
    if remove_duplicate_whitespace:
        return ' '.join(re.split('\s+', input_text.strip(), flags=re.UNICODE))
    return input_text.strip()

  def remove_stopword_method(self,input_text_or_list: Union[str, List[str]])-> List[str]:
    """ Remove stop words """

    stop_words = set(stopwords.words('english'))
    if isinstance(stop_words, list):
        stop_words = set(stop_words)
    if isinstance(input_text_or_list, str):
        tokens = word_tokenize(input_text_or_list)
        processed_tokens = [token for token in tokens if token not in stop_words]
    else:
        processed_tokens = [token for token in input_text_or_list
                            if (token not in stop_words and token is not None)]
    return processed_tokens
     
  def lemmatize_word_method(self,input_text_or_list: Union[str, List[str]])-> List[str]:
    """ Lemmatize each token in a text by finding its base form """
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    if isinstance(input_text_or_list, str):
        tokens = word_tokenize(input_text_or_list)
        processed_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    else:
        processed_tokens = [lemmatizer.lemmatize(token) for token in input_text_or_list if token is not None]
    return processed_tokens
     
  def tokenize_word_method(self,input_text: str) -> List[str]:
    """ Converts a text into a list of word tokens """
    if input_text is None:
        return []
    return word_tokenize(input_text)


  def tokenize_sentence_method(self,input_text: str) -> List[str]:
    """ Converts a text into a list of sentence tokens """
    if input_text is None:
        return []
    tokenizer = PunktSentenceTokenizer()
    return tokenizer.tokenize(input_text)

  def substitute_token_method(self,token_list: List[str])-> List[str]:
    """ Substitute each token by another token, e.g., 'vs' -> 'versus' """
    # TODO: add more custom substitutions in the csv file specified by _CUSTOM_SUB_CSV_FILE_PATH
    if token_list is None:
        return []
    with open(_CUSTOM_SUB_CSV_FILE_PATH, 'r') as f:
            csv_file = csv.reader(f)
            sub_dict = dict(csv_file)
    processed_tokens = list()

    for token in token_list:
        if token in sub_dict:
            processed_tokens.append(sub_dict[token])
        else:
            processed_tokens.append(token)
    return processed_tokens

  def check_spelling_method(self,input_text_or_list: Union[str, List[str]], lang='en',
                   ignore_word_file_path: Union[str, Path] = _IGNORE_SPELLCHECK_WORD_FILE_PATH) -> str:
    """ Check and correct spellings of the text list """
    if input_text_or_list is None:
        return ''
    spelling_checker = SpellChecker()
    # TODO: add acronyms into spell checker to ignore auto correction specified by _IGNORE_SPELLCHECK_WORD_FILE_PATH
    #spelling_checker.word_frequency.load_text_file(ignore_word_file_path)
    if isinstance(input_text_or_list, str):
        if not input_text_or_list.islower():
            input_text_or_list = input_text_or_list.lower()
        tokens = word_tokenize(input_text_or_list)
    else:
        tokens = [token.lower() for token in input_text_or_list if token is not None]
    misspelled = spelling_checker.unknown(tokens)
    for word in misspelled:
        tokens[tokens.index(word)] = spelling_checker.correction(word)
    return ' '.join(tokens).strip()

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
%%writefile config.py
to_lower = True
remove_url=True
remove_special_character=True
remove_punctuation=True
remove_whitespace=True
check_spelling=True
remove_stopword=True
substitute_token=True
lemmatize_word=True
tokenize_word=True
tokenize_sentence= False

Overwriting config.py


In [21]:
import config

In [22]:
config.tokenize_sentence check_spelling=config.check_spelling

False

In [34]:
pp = PreProcessing(to_lower = config.to_lower, remove_url=config.remove_url, remove_special_character=config.remove_special_character, 
                   remove_punctuation=config.remove_punctuation, remove_whitespace=config.remove_whitespace,
                   check_spelling = config.check_spelling, remove_stopword=config.remove_stopword, 
                   substitute_token=config.substitute_token, lemmatize_word=config.lemmatize_word,
                   tokenize_word=config.tokenize_word, tokenize_sentence= config.tokenize_sentence)

text_to_process = 'Helllo,    I am John Doe!!! My email is john.doe@email.com. Visit our website www.johndoe.com'

pp.preprocess(text_to_process)

FileNotFoundError: ignored

In [25]:
import os

_CUSTOM_SUB_CSV_FILE_PATH = os.path.join(os.getcwd(), 'data/custom_substitutions.csv')
_IGNORE_SPELLCHECK_WORD_FILE_PATH = os.path.join(os.getcwd(), 'data/ignore_spellcheck_words.txt')

_CUSTOM_SUB_CSV_FILE_PATH

'/content/data/custom_substitutions.csv'